In [ ]:
! pip install -r requirements.txt

In [1]:
from evaluator import Evaluator

ev = Evaluator(api_url=, api_key=, api_client)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


TypeError: Evaluator.__init__() missing 3 required positional arguments: 'api_url', 'api_key', and 'api_client'